In [ ]:
# Adding sequencing results to Y2H
# TODO: move somewhere else!
import numpy as np
import pandas as pd
import ccsblib
df = pd.read_csv('../data/internal/Y2H-data_2021-09-03_missing-sequence-confirmation.tsv', # 'internal/y2h_data.tsv',
                 sep='\t',
                 na_values=[''],
                 keep_default_na=False)
qry = """SELECT retest_pla, retest_pos, seq_confirmation as seq_confirmation_3AT
         FROM tf_screen.tffnl_scores;"""
seq_3at = pd.read_sql(qry, ccsblib.paros_connection())
df = pd.merge(df, 
    seq_3at, 
    on=['retest_pla', 'retest_pos'],
    how='left')
df['seq_confirmation_3AT'] = df['seq_confirmation_3AT'].map({'y': True, 'diff_isoform': False}).fillna(False)
# next line silently converts bool column to float
df.loc[(df['3AT'] == 'NA') | (df['3AT'] == '0'), 'seq_confirmation_3AT'] = np.nan
df['seq_confirmation_3AT'] = df['seq_confirmation_3AT'].astype('boolean')
qry = """SELECT retest_pla, retest_pos, 
                seq_confirmation as seq_confirmation_LW
         FROM tf_screen.tffnl2seq;"""
seq_lw = pd.read_sql(qry, ccsblib.paros_connection())
seq_lw['seq_confirmation_LW'] = seq_lw['seq_confirmation_LW'].map({'y': True, 'diff_isoform': False}).fillna(False)
df = pd.merge(df, 
    seq_lw, 
    on=['retest_pla', 'retest_pos'],
    how='left')
df['seq_confirmation_LW'] = df['seq_confirmation_LW'].fillna(True)
df.loc[(df['seq_confirmation_LW'] == False) | (df['seq_confirmation_3AT'] == False), 'Y2H_result'] = np.nan
df.to_csv('../data/internal/Y2H-data_2022-03-08.tsv', index=False, sep='\t')
